## Install Necessary Libraries

In [ ]:
!pip install mlflow tsfresh boto3 awscli raptor_functions

## AWS Configuration

In [ ]:
!aws configure

## Import Necessary Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')



import mlflow
from raptor_functions.supervised.datasets import get_data
from raptor_functions.supervised.prediction import get_prediction_features


## Load Data

In [2]:
df = get_data('handheld_data')
df.head()

,exp_unique_id,exp_name,timesteps,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,...,sensor_23,sensor_24,Humidity (r.h.),measurement_stage,date_exp,time_elapsed,datetime_exp_start,datetime_exp,filename,result
0,0,Test 24,1,131.938,132.443,177.444,178.157,44.974,44.508,219.374,...,143.288,142.050,46.07,baseline,21/3/2022,0.00,14:12:54,2022-03-21 14:12:54.000,5 NegTest_3_20220321_14_12.txt,Control
1,0,Test 24,2,133.078,133.972,177.444,178.694,45.131,44.611,219.607,...,142.874,141.914,46.07,baseline,21/3/2022,0.25,14:12:54,2022-03-21 14:12:54.250,5 NegTest_3_20220321_14_12.txt,Control
2,0,Test 24,3,131.812,132.064,177.622,178.694,45.183,44.663,219.841,...,143.288,141.914,46.07,baseline,21/3/2022,0.50,14:12:54,2022-03-21 14:12:54.500,5 NegTest_3_20220321_14_12.txt,Control
3,0,Test 24,4,131.059,132.064,177.444,178.694,45.183,44.663,219.607,...,142.599,141.368,46.07,baseline,21/3/2022,0.75,14:12:54,2022-03-21 14:12:54.750,5 NegTest_3_20220321_14_12.txt,Control
4,0,Test 24,5,131.435,131.938,177.622,178.694,45.131,44.663,219.607,...,142.187,140.825,46.07,baseline,21/3/2022,1.00,14:12:54,2022-03-21 14:12:55.000,5 NegTest_3_20220321_14_12.txt,Control


## Model Prediction

In [3]:
# assumed new data for prediction
x = df.groupby('exp_unique_id').get_group(0)

# Go to mlflow and copy relevant features from relevant_features tag of model to be used for prediciton
relevant_features = ['sensor_5__fft_coefficient__attr_"angle"__coeff_34', 'sensor_23__maximum']


data = get_prediction_features(x, relevant_features)


Feature Extraction: 100%|██████████| 33/33 [00:00<00:00, 6498.52it/s]


In [4]:

# use the full model path from s3 bucket (should begin with s3:// )
logged_model = 's3://raptor-mlflow-data/mlartifacts/1/99e55fa13b224817bdc47375af832289/artifacts/Random Forest Classifier'


# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
pred = loaded_model.predict(pd.DataFrame(data))

2022/04/13 00:19:21 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - psutil (current: 5.4.8, required: psutil==5.8.0)
 - scikit-learn (current: 1.0.2, required: scikit-learn==0.23.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [5]:
pred

array([1])

## To remove warning shown above

In [6]:
mlflow.pyfunc.get_model_dependencies(logged_model)

2022/04/13 00:19:36 INFO mlflow.pyfunc: To install these model dependencies, run the following command: 'pip install -r /tmp/tmpj1o7a7uq/requirements.txt'.


'/tmp/tmpj1o7a7uq/requirements.txt'

In [ ]:
# install model dependencies running the command shown above
!pip install -r /tmp/tmpdp46hihy/requirements.txt

In [7]:
pred = loaded_model.predict(pd.DataFrame(data))

In [8]:
pred

array([1])

## Load Model From Registry

In [ ]:
# %env MLFLOW_TRACKING_URI=http://ec2-3-10-175-206.eu-west-2.compute.amazonaws.com:5000


In [ ]:
# import os
# os.environ['MLFLOW_TRACKING_URI'] = http://ec2-3-10-175-206.eu-west-2.compute.amazonaws.com:5000

In [ ]:
# import mlflow.pyfunc

# model_name = "Logistic_Regression"
# model_version = 1

# model = mlflow.pyfunc.load_model(
#     model_uri=f"models:/{model_name}/{model_version}"
# )

# model.predict(data)